###### **Read the JSON  file as a Dataframe**

In [1]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-latest-news.json".
display(df)

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7b80dfed-1276-41db-8bf0-38f2cc6ed170)

###### **Selecting just the value column from the dataframe**

In [2]:
df = df.select("value")

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 4, Finished, Available)

In [3]:
display(df)

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4c0e251c-6e9d-46d8-8dcc-82dd75f6b7ee)

###### **Explode the JSON column**

In [4]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode(df["value"]).alias("json_object"))

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 6, Finished, Available)

In [5]:
display(df_exploded)

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3cdcc331-1c1c-4e08-b3da-bb7c1df8d002)

###### **Converting the Exploded JSON Dataframe to a single JSON string list**

In [6]:
json_list = df_exploded.toJSON().collect()

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 8, Finished, Available)

In [7]:
print(json_list[0])

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 9, Finished, Available)

{"json_object":{"about":[{"name":"Delicious in Dungeon","readLink":"https://api.bing.microsoft.com/api/v7/entities/500040b8-fef3-e066-9c4e-f4402ea99f0f"},{"name":"Kaiju No. 8","readLink":"https://api.bing.microsoft.com/api/v7/entities/0749f9d2-bf78-fcfb-7153-a58017c1f390"},{"name":"One Piece","readLink":"https://api.bing.microsoft.com/api/v7/entities/d9d472aa-98d8-4706-a779-7267ce706b7d"},{"name":"My Hero Academia","readLink":"https://api.bing.microsoft.com/api/v7/entities/2f353bb1-85fa-b748-84dc-562723b19c50"},{"name":"Mecha","readLink":"https://api.bing.microsoft.com/api/v7/entities/c2215650-d6d3-c6e2-25d0-fd1b299c548d"}],"category":"Entertainment","datePublished":"2024-05-20T09:41:00.0000000Z","description":"The 2024 anime lineup features seven standout new series, from the contemplative fantasy Frieren: Beyond Journey's End to the quirky cooking adventure","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=OVFT.HTLU6o4IbAcU-Vzlb_9S7y&pid=News","height":379,"width":700}}

In [8]:
import json

news_json = json.loads(json_list[25])

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 10, Finished, Available)

In [9]:
print(news_json)

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 11, Finished, Available)

{'json_object': {'category': 'World', 'datePublished': '2024-05-20T08:28:54.0000000Z', 'description': "Vietnam's National Assembly on Monday elected Tran Thanh Man as its new chairman, one of several key changes in an ongoing reshuffle of the country's top leadership.", 'mentions': [{'name': 'Vietnam'}, {'name': 'Speaker driver'}, {'name': 'Reuters'}], 'name': 'Vietnam parliament elects new speaker amid leadership reshuffle', 'provider': [{'_type': 'Organization', 'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=ODF.jFXbg3L7Ce_1pS4_IOR8CA&pid=news'}}, 'name': 'Reuters on MSN.com'}], 'url': 'https://www.msn.com/en-gb/news/world/vietnam-parliament-elects-new-speaker-amid-leadership-reshuffle/ar-BB1mHyuR'}}


In [10]:
print(news_json['json_object']['description'])

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 12, Finished, Available)

Vietnam's National Assembly on Monday elected Tran Thanh Man as its new chairman, one of several key changes in an ongoing reshuffle of the country's top leadership.


In [11]:
print(news_json['json_object']["name"])
print(news_json['json_object']["description"])
print(news_json['json_object']["category"])
print(news_json['json_object']["url"])

provider = news_json['json_object']['provider'][0]
if 'image' in provider and 'thumbnail' in provider['image']:
    print(provider['image']['thumbnail']['contentUrl'])
else:
    print("No image available")

print(news_json['json_object']["provider"][0]["name"])
print(news_json['json_object']["datePublished"])

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 13, Finished, Available)

Vietnam parliament elects new speaker amid leadership reshuffle
Vietnam's National Assembly on Monday elected Tran Thanh Man as its new chairman, one of several key changes in an ongoing reshuffle of the country's top leadership.
World
https://www.msn.com/en-gb/news/world/vietnam-parliament-elects-new-speaker-amid-leadership-reshuffle/ar-BB1mHyuR
https://www.bing.com/th?id=ODF.jFXbg3L7Ce_1pS4_IOR8CA&pid=news
Reuters on MSN.com
2024-05-20T08:28:54.0000000Z


###### **Processing the JSON property to List**

In [12]:
title = []
description = []
category = []
url = []
image = []
provider = []
datePublished = []

for json_str in json_list:
    try:
        article = json.loads(json_str)  # Parse the JSON string into a dictionary

        if article["json_object"].get("category"):

            title.append(article['json_object']['name'])
            description.append(article['json_object']['description'])
            category.append(article['json_object']['category'])
            url.append(article['json_object']['url'])

            provider_data = article['json_object'].get('provider', [{}])[0]
            image_url = provider_data.get('image', {}).get('thumbnail', {}).get('contentUrl', 'No image available')
            image.append(image_url)
            
            provider.append(article['json_object']["provider"][0]["name"])
            datePublished.append(article['json_object']['datePublished'])

    except Exception as e:
        print(f"Error processing JSON object: {e}")

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 14, Finished, Available)

###### **Converting the List to a Dataframe**

In [13]:
from pyspark.sql.types import StructType, StructField, StringType

#combine the lists
data = list(zip(title,description,category,url,image,provider,datePublished))

#define schema
schema = StructType([
    StructField("title", StringType(),True),
    StructField("description", StringType(),True),
    StructField("category", StringType(),True),
    StructField("url", StringType(),True),
    StructField("image", StringType(),True),
    StructField("provider", StringType(),True),
    StructField("datePublished", StringType(),True),

])

df_cleaned = spark.createDataFrame(data, schema=schema)

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 15, Finished, Available)

In [14]:
display(df_cleaned)

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, ad6f6cbb-e5b1-44fc-83d5-5067216273f4)

In [15]:
from pyspark.sql.functions import to_date, date_format

df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 17, Finished, Available)

In [16]:
display(df_cleaned_final.limit(5))

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1ab48d56-9c55-4f43-ac80-461136c92245)

###### **Writing the Final Dataframe to the Lakehouse DB in a Delta format**

In [19]:
df_cleaned_final.write.format("delta").saveAsTable("lake_db.tbl_latest")

StatementMeta(, 41889772-737d-41bd-b04d-e6ad67daee1d, 21, Finished, Available)